In [1]:
import numpy as np
import qutip
import pennylane as qml
import pandas as pd


n_layers = 2
n_wires = 4

iteracoes = 3

def StronglyEntanglingLayers(n_layers, n_wires):

    #Cria dados
    def normaliza(v):
        v = v / (np.linalg.norm(v))    
        return v

    def cria_alvos():
        target_states = [normaliza(np.random.rand(1,16)), 
                        normaliza(np.random.rand(1,16)),
                        normaliza(np.random.rand(1,16)),
                        normaliza(np.random.rand(1,16)),
                        normaliza(np.random.rand(1,16)),
                        normaliza(np.random.rand(1,16)), 
                        normaliza(np.random.rand(1,16)),
                        normaliza(np.random.rand(1,16)),
                        normaliza(np.random.rand(1,16)),
                        normaliza(np.random.rand(1,16))]    
        return target_states
    #print("Estados alvo: {}".format(cria_alvos()))

    def cria_pesos(n_layers, n_wires):
        shape = qml.StronglyEntanglingLayers.shape(n_layers, n_wires)   #Formato do vetor de matrizes
        weights = np.random.random(size=shape)
        #print("\nVetor de matrizes peso: \n {}".format(weights))
        #print("\n")
        return weights



    #Cria circuitos
    @qml.qnode(dev)
    def circuit1(parameters):
        qml.StronglyEntanglingLayers(weights=parameters, wires=range(4))
        return qml.state()

    @qml.qnode(dev)
    def circuit2(parameters):
        qml.StronglyEntanglingLayers(weights=parameters, wires=range(4))
        return qml.probs(wires=range(4))



    #Instanciação
    weights = cria_pesos(n_layers, n_wires)
    print(qml.draw(circuit1, expansion_strategy="device")(weights))
    print("\n")
    ket = circuit1(weights)
    amplt = circuit2(weights)
    target_states = cria_alvos()

    #print("\n", ket)
    #print("\n", amplt)


    #Expressabilidade
    def fidelity(target_states, amplt):
        fidelity_sum = 0
        for target_state in target_states:
            output_probs = amplt
            fidelity = np.vdot(target_state, output_probs) ** 2
            fidelity_sum += fidelity
    
        average_fidelity = fidelity_sum / len(target_states)
        return average_fidelity



    #Grau de Emaranhamento
    def compute_Q_ptrace(ket, N):
        ket = qutip.Qobj(ket, dims=[[2]*(N), [1]*(N)]).unit()
        entanglement_sum = 0
        for k in range(N):
            rho_k_sq = ket.ptrace([k])**2
            entanglement_sum += rho_k_sq.tr()  
   
        Q = 2*(1 - (1/N)*entanglement_sum)
        return Q



    Q = compute_Q_ptrace(ket, n_wires)
    #print("\nGrau de Emaranhamento: {}".format(Q))

    E = fidelity(target_states, amplt)
    #print("Expressabilidade: {}".format(E))

    #print("\n\nEstado gerado: {}".format(ket))
    #print("\n\n\n\n\n")

    k=[Q,E]


    return k


# qml.probs retorna a probabilidade de obter como resultado cada base computacional
# Para 4 qubits este nos retornará 2^4 = 16 valores
# Para transformar o objeto qml.probs() em qml.state() precisamos calcular a norma ao quadrado de cada componente do vetor

dev = qml.device('default.qubit', wires=n_wires)



k=[]
for i in range(iteracoes):
    print("=============")
    print("= Modelo: {} =".format(i+1))
    print("=============\n")
    k.append(StronglyEntanglingLayers(n_layers, n_wires))


print("\n\nResultados:\n")
for i in range(iteracoes):
    print(" Modelo {} \n Grau de emaranhamento: {} \n Expressabilidade: {} \n".format(i+1,k[i][0],k[i][1]))


def pand(k):
    eman=[]
    exp=[]

    for i in range(iteracoes):
        eman.append(k[i][0])
    for i in range(iteracoes):
        exp.append(k[i][1]) 

    dic = {"Grau de Emaranhamento": eman, "Expressabilidade":exp}
    df = pd.DataFrame(dic)
    return df

df = pand(k)
df.describe()


= Modelo: 1 =

0: ──Rot(0.43,0.80,0.91)─╭●───────╭X──Rot(0.78,0.47,0.54)─╭●────╭X────┤  State
1: ──Rot(0.76,0.75,0.51)─╰X─╭●────│───Rot(0.63,0.28,0.52)─│──╭●─│──╭X─┤  State
2: ──Rot(0.84,0.92,0.22)────╰X─╭●─│───Rot(0.80,0.75,0.72)─╰X─│──╰●─│──┤  State
3: ──Rot(0.05,0.45,0.95)───────╰X─╰●──Rot(0.08,0.18,0.94)────╰X────╰●─┤  State


= Modelo: 2 =

0: ──Rot(0.47,0.46,0.30)─╭●───────╭X──Rot(0.94,1.00,0.64)─╭●────╭X────┤  State
1: ──Rot(0.16,1.00,0.14)─╰X─╭●────│───Rot(0.62,0.61,0.15)─│──╭●─│──╭X─┤  State
2: ──Rot(0.57,0.14,0.07)────╰X─╭●─│───Rot(0.06,0.40,0.73)─╰X─│──╰●─│──┤  State
3: ──Rot(0.92,0.04,0.12)───────╰X─╰●──Rot(0.04,0.99,0.98)────╰X────╰●─┤  State


= Modelo: 3 =

0: ──Rot(0.68,0.57,0.16)─╭●───────╭X──Rot(0.77,0.67,0.54)─╭●────╭X────┤  State
1: ──Rot(0.35,0.74,0.06)─╰X─╭●────│───Rot(0.96,0.45,0.93)─│──╭●─│──╭X─┤  State
2: ──Rot(0.52,0.12,0.90)────╰X─╭●─│───Rot(0.77,0.65,0.17)─╰X─│──╰●─│──┤  State
3: ──Rot(0.50,0.89,0.21)───────╰X─╰●──Rot(0.22,0.17,0.48)────╰X────╰●─┤  State





,Grau de Emaranhamento,Expressabilidade
count,3.000000,3.000000
mean,0.734315,0.047824
std,0.064125,0.004869
min,0.662634,0.042319
25%,0.708357,0.045953
50%,0.754081,0.049588
75%,0.770155,0.050576
max,0.786230,0.051564
